# Docker Tutorial

- Docker is a free open platform to containerize package for reproduction

  - Docker: Buleprint to build image
  - Image: Template to run container
  - Container: Running package

## Install

- Docker Desktop WSL 2 backend on Windows
- https://docs.docker.com/desktop/windows/wsl/

Create python requirements.txt

In [3]:
%%writefile requirements.txt
numpy==1.24.1

Overwriting requirements.txt


Create environment
```
conda create --name docker-env python=3.8 --file requirements.txt
conda activate docker-env

```
To run notebook in VSCode
```
conda install notebook ipykernel
```

Start VSCode
```
code .
```

In [1]:
# docker version

!docker -v

Docker version 20.10.22, build 3a2c30b


In [28]:
# docker detail

!docker info

Client:
 Context:    default
 Debug Mode: false
 Plugins:
  buildx: Docker Buildx (Docker Inc., v0.10.0)
  compose: Docker Compose (Docker Inc., v2.15.1)
  dev: Docker Dev Environments (Docker Inc., v0.0.5)
  extension: Manages Docker extensions (Docker Inc., v0.2.17)
  sbom: View the packaged-based Software Bill Of Materials (SBOM) for an image (Anchore Inc., 0.6.0)
  scan: Docker Scan (Docker Inc., v0.23.0)

Server:
 Containers: 3
  Running: 0
  Paused: 0
  Stopped: 3
 Images: 4
 Server Version: 20.10.22
 Storage Driver: overlay2
  Backing Filesystem: extfs
  Supports d_type: true
  Native Overlay Diff: true
  userxattr: false
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Cgroup Version: 1
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: io.containerd.runc.v2 io.containerd.runtime.v1.linux runc
 Default Runtime: runc
 Init Binary: docker

In [2]:
# test docker

!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Prepare files


In [3]:
%%writefile main.py
import numpy as np

pi_value = np.pi
print(f"pi = {pi_value}")

Overwriting main.py


In [4]:
%%writefile Dockerfile
# For more information, please refer to https://aka.ms/vscode-docker-python
FROM python:3.8-slim

# Keeps Python from generating .pyc files in the container
ENV PYTHONDONTWRITEBYTECODE=1

# Turns off buffering for easier container logging
ENV PYTHONUNBUFFERED=1

# Install pip requirements
COPY requirements.txt .
RUN python -m pip install -r requirements.txt

WORKDIR /app
COPY . /app

# Creates a non-root user with an explicit UID and adds permission to access the /app folder
# For more info, please refer to https://aka.ms/vscode-docker-python-configure-containers
RUN adduser -u 5678 --disabled-password --gecos "" appuser && chown -R appuser /app
USER appuser

# During debugging, this entry point will be overridden. For more information, please refer to https://aka.ms/vscode-docker-python-debug
CMD ["python", "main.py"]

Overwriting Dockerfile


In [5]:
%%writefile .dockerignore
**/__pycache__
**/.venv
**/.classpath
**/.dockerignore
**/.env
**/.git
**/.gitignore
**/.project
**/.settings
**/.toolstarget
**/.vs
**/.vscode
**/*.*proj.user
**/*.dbmdl
**/*.jfm
**/bin
**/charts
**/docker-compose*
**/compose*
**/Dockerfile*
**/node_modules
**/npm-debug.log
**/obj
**/secrets.dev.yaml
**/values.dev.yaml
LICENSE
README.md

Overwriting .dockerignore


Alternatively, we can use docker extension to create Dockerfile

- Ctl + Shift + P
- Choose Docker: Add Docker Files to Workspace
- Choose Python General
- Choose .py
- Choose No for Docker compose files

### Create image

In [37]:
# create python-pi image
!docker build -t python-pi .

```
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 876B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 382B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         0.1s
 ...
  => [6/6] RUN adduser -u 5678 --disabled-password --gecos "" appuser && c  0.8s
 => exporting to image                                                     0.5s
 => => exporting layers                                                    0.5s
 => => writing image sha256:e599ae4bd3e7348959fd6f18d8add2bdbc05e04f9ebf3  0.0s
 => => naming to docker.io/library/python-pi                               0.0s

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them
```

In [8]:
# Check all image 
!docker image list

REPOSITORY          TAG       IMAGE ID       CREATED             SIZE
python-pi           latest    e599ae4bd3e7   17 minutes ago      214MB
docker101tutorial   latest    9aea348160d1   About an hour ago   47MB
alpine/git          latest    22d84a66cda4   2 months ago        43.6MB
hello-world         latest    feb5d9fea6a5   16 months ago       13.3kB


In [7]:
# docker run
# used to create a container from an image
!docker run python-pi

pi = 3.141592653589793


In [9]:
# docker ps
# list all the running containers in the background.

!docker ps

CONTAINER ID   IMAGE               COMMAND                  CREATED             STATUS             PORTS                NAMES
2ae3b6c4ab00   docker101tutorial   "/docker-entrypoint.…"   About an hour ago   Up About an hour   0.0.0.0:80->80/tcp   docker-tutorial


In [10]:
# Show run history

!docker ps --all

CONTAINER ID   IMAGE               COMMAND                  CREATED             STATUS                      PORTS                NAMES
fede5aeabeb2   python-pi           "python main.py"         19 minutes ago      Exited (0) 19 minutes ago                        wonderful_torvalds
6533a116a42e   hello-world         "/hello"                 20 minutes ago      Exited (0) 20 minutes ago                        amazing_hertz
2ae3b6c4ab00   docker101tutorial   "/docker-entrypoint.…"   About an hour ago   Up About an hour            0.0.0.0:80->80/tcp   docker-tutorial


In [15]:
# docker stop
# stops a container using the container name or its id. 

!docker stop 2ae3b6c4ab00

2ae3b6c4ab00


In [16]:
# docker restart
# This command is used to restart the stopped container. 
# It is recommended to use this after rebooting the system. 

!docker restart 2ae3b6c4ab00

2ae3b6c4ab00


In [17]:
# docker kill
# This command is used to stop the container immediately by killing its execution. 

!docker kill 2ae3b6c4ab00


2ae3b6c4ab00


In [19]:
# docker commit
# to create or save an image of the edited container on the local system.

!docker commit fede5aeabeb2 test

sha256:443c23d28e674d64854141aaf3e69765332b9d9fc7d6139e266c73e8fbcc4741


In [31]:
# docker cp
# copy a file from a docker container to the local system.

!docker cp e599ae4bd3e7:/usr/local/apache2/logs/main.py /home/jingwora/docker-tutorial/temp

Error: No such container:path: e599ae4bd3e7:/usr/local/apache2/logs/main.py


In [22]:
# confirm the copy 
!docker image list

REPOSITORY          TAG       IMAGE ID       CREATED              SIZE
test                latest    443c23d28e67   About a minute ago   214MB
python-pi           latest    e599ae4bd3e7   46 minutes ago       214MB
docker101tutorial   latest    9aea348160d1   2 hours ago          47MB
alpine/git          latest    22d84a66cda4   2 months ago         43.6MB
hello-world         latest    feb5d9fea6a5   16 months ago        13.3kB


In [25]:
# docker rmi 
# remove image

!docker rmi 443c23d28e67

Untagged: test:latest
Deleted: sha256:443c23d28e674d64854141aaf3e69765332b9d9fc7d6139e266c73e8fbcc4741


In [24]:
# docker rm
# remove docker container
# docker rm eea5b2620e02 5200866fb18d # many files

!docker rm 443c23d28e67

In [ ]:
# docker exec
# access the docker container and run commands inside the container
!docker exec -it 09ca6feb6efc bash

In [32]:
# docker history
# shows the history of a docker image with the image name mentioned in the command.

!docker history python-pi

IMAGE          CREATED             CREATED BY                                      SIZE      COMMENT
e599ae4bd3e7   About an hour ago   CMD ["python" "main.py"]                        0B        buildkit.dockerfile.v0
<missing>      About an hour ago   USER appuser                                    0B        buildkit.dockerfile.v0
<missing>      About an hour ago   RUN /bin/sh -c adduser -u 5678 --disabled-pa…   2.88MB    buildkit.dockerfile.v0
<missing>      About an hour ago   COPY . /app # buildkit                          1.03MB    buildkit.dockerfile.v0
<missing>      About an hour ago   WORKDIR /app                                    0B        buildkit.dockerfile.v0
<missing>      About an hour ago   RUN /bin/sh -c python -m pip install -r requ…   85.8MB    buildkit.dockerfile.v0
<missing>      About an hour ago   COPY requirements.txt . # buildkit              14B       buildkit.dockerfile.v0
<missing>      About an hour ago   ENV PYTHONUNBUFFERED=1                          0B  

In [35]:
# container ls
# list container

!docker container ls

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [36]:
# docker logs
# show the logs of the docker container with contained id mentioned in the command.

!docker logs e599ae4bd3e7

In [27]:
# docker network ls
# details of the list of networks in the cluster.

!docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
6ab1ecfb9007   bridge    bridge    local
9e732aebb3f6   host      host      local
4fc177c2ecdb   none      null      local


### Docker Hub

In [ ]:
# docker login
# log in to your docker hub

!docker login

In [ ]:
# docker logout
# log you out of the docker hub.

!docker logout

In [12]:
# docker search
# searches for specific images through the Docker hub. 

!docker search MySQL

NAME                            DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
mysql                           MySQL is a widely used, open-source relation…   13733     [OK]       
mariadb                         MariaDB Server is a high performing open sou…   5241      [OK]       
phpmyadmin                      phpMyAdmin - A web interface for MySQL and M…   726       [OK]       
percona                         Percona Server is a fork of the MySQL relati…   599       [OK]       
databack/mysql-backup           Back up mysql databases to... anywhere!         80                   
bitnami/mysql                   Bitnami MySQL Docker Image                      80                   [OK]
linuxserver/mysql-workbench                                                     48                   
ubuntu/mysql                    MySQL open source fast, stable, multi-thread…   41                   
linuxserver/mysql               A Mysql container, brought to you by 

In [ ]:
# docker pull
# pulls a specific image from the Docker Hub.

!docker pull --platform Linux/x86_64 MySQL

In [ ]:
# docker push
# push or upload a docker image on the repository or the docker hub. 

!docker push python-pi

Ref:
- https://hub.docker.com/ 
- https://biocontainers.pro/
- https://docs.docker.com/engine/reference/run/